In [ ]:
# !pip install glob2
# !pip install pathlib
import glob
import os
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def sectionChecker(directory):
  
    # opening a text file
    file = open(directory, "r")
    if not os.path.exists('/content/drive/MyDrive/Torque and Drag Training Data Files'):
      os.makedirs('/content/drive/MyDrive/Torque and Drag Training Data Files')
    fileOutput = open('/content/drive/MyDrive/Torque and Drag Training Data Files/FilesContainingSections.txt', 'a')
    outputDirectory = '/content/drive/MyDrive/Torque and Drag Training Data Files/Volve-'+(directory.split('/')[5])+'--'
    sections=[]
    sectionFileList = []

    # Loop through the file line by line
    for line in file:  

        # checking for sections specified regardless of case sensitivity
        if ('8.5 in ' in line.casefold() or '8.5in ' in line.casefold() or '8.5" ' in line or '8.5 " ' in line) and '8.5' not in sections:
            sections.append('8.5')
            outputDirectory = outputDirectory + '8.5in--'
            # print('8.5!')
        if ('12.25 in ' in line.casefold() or '12.25in ' in line.casefold() or '12.25" ' in line or '12.25 " ' in line) and '12.25' not in sections:
            sections.append('12.25')
            outputDirectory = outputDirectory + '12.25in--'
            # print('12.25!')
        if ('17.5 in ' in line.casefold() or '17.5in ' in line.casefold() or '17.5" ' in line or '17.5 " ' in line) and '17.5' not in sections:
            sections.append('17.5')
            outputDirectory = outputDirectory + '17.5in--'
    outputDirectory = outputDirectory + 'Torque and Drag Training Data File.csv'
    if '8.5' in outputDirectory or '12.25' in outputDirectory or '17.5' in outputDirectory:
      df=pd.read_csv(directory)
      comma_separated_data=df["data"]
      comma_separated_mnemonics=df["mnemonicList"]
      comma_separated_units=df['unitList']
      colsList=[header for header in comma_separated_mnemonics[0].split(',')]
      unitsList = [unit for unit in comma_separated_units[0].split(',')]

      intermediateDf=[]
      for i in range(len(comma_separated_data)):
        if type(comma_separated_data[i]) != float:
          row=[entry for entry in comma_separated_data[i].split(',') ]                    #looks for the specific text we are looking for in the lines and add them to a list
          intermediateDf.append(row)
      requiredColumns = ['TIME', 'TFLO', 'RPM', 'DMEA', 'ROP', 'DBTM', 'TDH', 'HKLD', 'SWOB', 'TQA', 'SPPA']
      intermediateColumnsTwo = []
      intermediateDfTwo = []
      finalDf = [[]]
      finalColumns = []
      intermediateUnits = []
      finalUnits = []
      for i in range(len(requiredColumns)):
        for j in range(len(colsList)):
          if requiredColumns[i].casefold() == colsList[j].casefold() or "gs_"+requiredColumns[i].casefold() == colsList[j].casefold():
            intermediateColumnsTwo.append(colsList[j])
            intermediateUnits.append(unitsList[j])
            for k in range(len(intermediateDf)):
              if len(intermediateDfTwo) < len(intermediateDf):
                intermediateDfTwo.append([])
              intermediateDfTwo[k].append(intermediateDf[k][j])
      while len(finalDf)< len(intermediateDfTwo):
        finalDf.append([])
      for column in intermediateColumnsTwo:
        index = intermediateColumnsTwo.index(column)
        for columns in requiredColumns:
          if column.casefold() == columns.casefold() and "GS_"+columns in intermediateColumnsTwo:
            indexGS = intermediateColumnsTwo.index("GS_" + column)
            for l in range (len(intermediateDfTwo)): 
                if intermediateDfTwo[l][indexGS] == '':
                  finalDf[l].append(intermediateDfTwo[l][index])
                elif intermediateDfTwo[l][index] == '':
                  finalDf[l].append(intermediateDfTwo[l][indexGS])
            finalColumns.append(column)
            finalUnits.append(intermediateUnits[intermediateColumnsTwo.index(column)])
          elif column.casefold() == columns.casefold() and "GS_"+columns not in intermediateColumnsTwo:
            for m in range(len(intermediateDfTwo)):
              finalDf[m].append(intermediateDfTwo[m][index])
            finalColumns.append(column)
            finalUnits.append(intermediateUnits[intermediateColumnsTwo.index(column)])
          elif column.casefold() == "gs_"+columns.casefold() and columns not in intermediateColumnsTwo:
            finalColumns.append(columns)
            finalUnits.append(intermediateUnits[requiredColumns.index(columns)])
            for n in range (len(intermediateDfTwo)):
              finalDf[n].append(intermediateDfTwo[n][index])
      # for i in range(len(finalColumns)):
      #   finalColumns[i] = finalColumns[i] + " (" + finalUnits[i] + ")"
      df2 = pd.DataFrame(finalDf, columns = finalColumns)
      outputFileAdjust = outputDirectory.replace('.csv', '') + " 0000" + '.csv'
      while Path(outputFileAdjust).is_file():
        outputFileAdjust = outputFileAdjust.replace('.csv', '')
        outputFileNumber = outputFileAdjust[-4:]
        outputFileAdjust = outputFileAdjust[:-4]
        outputFileAdjust = outputFileAdjust + str(int(outputFileNumber)+1).zfill(4) + '.csv'
      df2.to_csv(outputFileAdjust)
    writeStr = str(directory) + ' ' + str(sections) + '\n'
    if sections!=[]:
        fileOutput.write(writeStr)


    # checking condition for string found or not

    # closing text file    
    file.close() 
    fileOutput.close()

In [ ]:
# fileOutput = open('/content/drive/MyDrive/Torque and Drag Training Data Files/FilesContainingSections.txt', 'w')
# fileOutput.write('')
# fileOutput.close()
for i in (Path("/content/drive/MyDrive/ExtractedWITSMLData/").rglob(('*.csv'))): #searches DrillingData folder containing all WITSML realtime drilling data (renamed from WITSML Realtime drilling data). DrillingData folder must be in the same directory as this code.
    if 'log' in str(i):
      print(i, 'beginning.')
      sectionChecker(str(i)) #sends file path of discovered .xml file to dataExtractor function.
      print(i, 'complete.')
print ('Section checking complete.')

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-1 C/4/log/1/4/3/00007.csv complete.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-1 C/4/log/1/4/3/00008.csv beginning.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-1 C/4/log/1/4/3/00008.csv complete.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-1 C/4/log/1/4/3/00009.csv beginning.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-1 C/4/log/1/4/3/00009.csv complete.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-1 C/4/log/1/4/3/00010.csv beginning.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-1 C/4/log/1/4/3/00010.csv complete.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-1 C/4/log/1/4/3/00011.csv beginning.
/content/drive/MyDrive/ExtractedWITSMLData/Norway-Statoil-NO 15_$47$_9-F-

In [ ]:
dataframeDrilling = []
dataframeDrillingTwo = []
dataframeDrillingThree = []
dataframeNotDrilling = []
dataframeNotDrillingTwo = []
dataframeNotDrillingThree = []
wellNames = []
for file in (Path("/content/drive/MyDrive/Torque and Drag Training Data Files/").rglob(('*.csv'))):
    wellName = str(file).split('--')[0]
    if wellName not in wellNames:
      wellNames.append(wellName)
for well in wellNames:
  dfWells=[]
  for file in (Path("/content/drive/MyDrive/Torque and Drag Training Data Files/").rglob(('*.csv'))):
    if (str(file).split('--')[0]) == well:
      section = str(file).split('--')[1]
      df = pd.read_csv(str(file))
      if 'TIME' in df.columns and len(df.columns) == 12:
        df = df.reindex(sorted(df.columns), axis=1)
        df = df.drop('Unnamed: 0', axis=1)
        df['TIME'] = df['TIME'].str.replace('Z', '000')
        df = df.replace(to_replace = '-999.25', value =-999.25)
        df = df.replace(to_replace = -999.25, value =np.nan)
        df = df.interpolate('linear')
        df.columns = ['DBTM (m)', 'DMEA (m)', 'HKLD (t)', 'ROP (m/h)', 'RPM (rpm)', 'SPPA (kPa)', 'SWOB (t)', 'TDH (degC)', 'TFLO (L/min)','TIME', 'TQA (kN/m)']
        df['Section']=section
        df = df.set_index('TIME')
        df.index = pd.to_datetime(df.index, format = '%Y-%m-%d %H:%M:%S.%f')  
        dfWells.append(df)
  dfWell = pd.concat(dfWells)
  dfWell.sort_index(inplace=True)  
  # if not os.path.exists('/content/drive/MyDrive/Concatenated Logs'):
  #   os.makedirs('/content/drive/MyDrive/Concatenated Logs')
  dfWell.to_csv(well +'.csv') 
      
      
    #     for g in range(len(df.index)):
    #       if list(df.iloc[g])[3] > 0 and list(df.iloc[g])[4] > 0 and list(df.iloc[g])[6] > 0 and list(df.iloc[g])[0] == list(df.iloc[g])[1]:
    #         if '8.5' in str(file):
    #           dataframeDrilling.append(list(df.iloc[g]))
    #         elif '12.25' in str(file):
    #           dataframeDrillingTwo.append(list(df.iloc[g]))
    #         elif '17.5' in str(file):
    #           dataframeDrillingThree.append(list(df.iloc[g]))
            
    #       else:
    #         if '8.5' in str(file):
    #           dataframeNotDrilling.append(list(df.iloc[g]))
    #         elif '12.25' in str(file):
    #           dataframeNotDrillingTwo.append(list(df.iloc[g]))
    #         elif '17.5' in str(file):
    #           dataframeNotDrillingThree.append(list(df.iloc[g]))
    # del g
    # del file
  #   columnsforOutput = ['DBTM (m)', 'DMEA (m)', 'HKLD (t)', 'ROP (m/h)', 'RPM (rpm)', 'SPPA (kPa)', 'SWOB (t)', 'TDH (degC)', 'TFLO (L/min)','TIME', 'TQA (kN/m)']
  #   if len(dataframeDrilling) != 0:
  #     df = pd.DataFrame(dataframeDrilling, columns = columnsforOutput)
  #     df = df.set_index('TIME')
  #     df.index = pd.to_datetime(df.index, format = '%Y-%m-%d %H:%M:%S.%f')
  #     df.sort_index(inplace=True)                                 #sort the index by depth                             #linearly interploates the missing values
  #     df.reset_index(inplace=True)                                #Modify the Series in place (do not create a new object).
  #     df = df.rename(columns = {'index':'TIME (s)'})
  #     df.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/8.5in Drilling.csv')                       #saves the dataframe as with the first argument you passed as arguement to the function
  #   del dataframeDrilling

  #   if len(dataframeDrillingTwo) != 0:
  #     df = pd.DataFrame(dataframeDrillingTwo, columns = columnsforOutput)
  #     df = df.set_index('TIME')
  #     df.index = pd.to_datetime(df.index, format = '%Y-%m-%d %H:%M:%S.%f')
  #     df.sort_index(inplace=True)                                 #sort the index by depth                              #linearly interploates the missing values
  #     df.reset_index(inplace=True)                                #Modify the Series in place (do not create a new object).
  #     df = df.rename(columns = {'index':'TIME (s)'})
  #     df.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/12.25in Drilling.csv')

  #   del dataframeDrillingTwo
  #   if len(dataframeDrillingThree) != 0:
  #     df = pd.DataFrame(dataframeDrillingThree, columns=columnsforOutput)
  #     df = df.set_index('TIME')
  #     # df = df.drop(df.columns[0], axis=1)
  #     df.index = pd.to_datetime(df.index, format = '%Y-%m-%d %H:%M:%S.%f')
  #     df.sort_index(inplace=True)                                 #sort the index by depth
  #     # df = df.replace(to_replace=999999.25, value=np.nan)
  #     # df = df.interpolate('linear')                               #linearly interploates the missing values
  #     df.reset_index(inplace=True)                                #Modify the Series in place (do not create a new object).
  #     df = df.rename(columns = {'index':'TIME (s)'})
  #     df.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/17.5in Drilling.csv')

  #   del dataframeDrillingThree

  #   if len(dataframeNotDrilling) != 0:
  #     df = pd.DataFrame(dataframeNotDrilling, columns = columnsforOutput)
  #     df = df.set_index('TIME')
  #     # df = df.drop(df.columns[0], axis=1)
  #     df.index = pd.to_datetime(df.index, format = '%Y-%m-%d %H:%M:%S.%f')
  #     df.sort_index(inplace=True)                                 #sort the index by depth                        #linearly interploates the missing values
  #     df.reset_index(inplace=True)                                #Modify the Series in place (do not create a new object).
  #     df = df.rename(columns = {'index':'TIME (s)'})
  #     df.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/8.5in Not Drilling.csv')

  #   del dataframeNotDrilling

  #   if len(dataframeNotDrillingTwo) != 0:
  #     df = pd.DataFrame(dataframeNotDrillingTwo, columns = columnsforOutput)
  #     df = df.set_index('TIME')
  #     df.index = pd.to_datetime(df.index, format = '%Y-%m-%d %H:%M:%S.%f')
  #     df.sort_index(inplace=True)                                 #sort the index by depth                              #linearly interploates the missing values
  #     df.reset_index(inplace=True)                                #Modify the Series in place (do not create a new object).
  #     df = df.rename(columns = {'index':'TIME (s)'})
  #     df.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/12.25in Not Drilling.csv')

  #   del dataframeNotDrillingTwo

  #   if len(dataframeNotDrillingThree) != 0:
  #     df = pd.DataFrame(dataframeNotDrillingThree, columns = columnsforOutput)
  #     df = df.set_index('TIME')
  #     df.index = pd.to_datetime(df.index, format = '%Y-%m-%d %H:%M:%S.%f')
  #     df.sort_index(inplace=True)                                 #sort the index by depth
  #     df.reset_index(inplace=True)                                #Modify the Series in place (do not create a new object).
  #     df = df.rename(columns = {'index':'TIME (s)'})
  #     df.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/17.5in Not Drilling.csv')

  #   del dataframeNotDrillingThree

  #   print('Combined spreadsheet output complete.')
  # # i = 0

    
  # # for dataframe in dataframes:
  # #   if i == 0:
  # #     dataframe.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/IntermediateCombined.csv')
  # #     i+=1
  # #   if i > 0:
  # #     dataframe.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/IntermediateCombined.csv', mode='a', header=False)
  # # del dataframes
  # # finalDataframe = pd.read_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/IntermediateCombined.csv')
  # # finalDataframe.index = pd.to_datetime(finalDataframe.index, infer_datetime_format=True, errors='coerce')
  # # finalDataframe.sort_index(inplace=True)
  # # finalDataframe = finalDataframe.interpolate('linear')
  # # finalDataframe.to_csv('/content/drive/MyDrive/Torque and Drag Training Data Files/Combined.csv')
  # # print('Combined spreadsheet outputted.')

In [ ]:
for log in (Path("/content/drive/MyDrive/VolveWellLogs/").rglob(('*.csv'))):
  wellName = (str(log).split('/')[5]).replace('.csv', '').replace('Volve-','')
  for trajectory in (Path("/content/drive/MyDrive/VolveTrajectoryLogs/").rglob(('*.csv'))):
    wellNameTrajectory = (str(trajectory).split('/')[5]).replace('.csv', '')
    if wellNameTrajectory == wellName:
      dfLog = pd.read_csv(str(log))
      dfTraj = pd.read_csv(str(trajectory))
      /



Norway-StatoilHydro-15_$47$_9-F-10
Norway-StatoilHydro-15_$47$_9-F-15
Norway-StatoilHydro-15_$47$_9-F-15A
Norway-StatoilHydro-15_$47$_9-F-15B
Norway-StatoilHydro-15_$47$_9-F-15S
Norway-StatoilHydro-15_$47$_9-F-9
Norway-NA-15_$47$_9-F-9 A
